In [76]:
import create_vocab
import data_to_tensors
import model_implementation
from train_class import TrainingModule
import torch
import torch.nn as nn
import torch.nn.functional as F
import random 

from torch.utils.data import DataLoader

In [82]:
# data_folder = 'Codes_embedding'
# dataset_name = 'Codes_embedding'
data_folder = 'CODES_emb'
dataset_name = 'CODES_emb'
dict_path = 'data/'+ data_folder + '/' + dataset_name + '.dict.c2v'
word2idx, path2idx, target2idx, idx2target = create_vocab.create_vocab(dict_path)

In [78]:
path_for_train = 'data/'+ data_folder + '/' + dataset_name + '.train.c2v'
train_dataset = data_to_tensors.TextDataset(path_for_train, 
                                                    word2idx, 
                                                    path2idx, 
                                                    target2idx)

path_for_val = 'data/'+ data_folder + '/' + dataset_name + '.val.c2v'
val_dataset = data_to_tensors.TextDataset(path_for_val, 
                                                    word2idx, 
                                                    path2idx, 
                                                    target2idx)

path_for_test = 'data/'+ data_folder + '/' + dataset_name + '.test.c2v'
test_dataset = data_to_tensors.TextDataset(path_for_test, 
                                                    word2idx, 
                                                    path2idx, 
                                                    target2idx)

train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=512, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=512, shuffle=False)

# Train

In [79]:
SEED = 1337
random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [80]:
# In case if not bert

model = model_implementation.code2vec_model(values_vocab_size = len(word2idx), 
                                paths_vocab_size = len(path2idx), 
                                labels_num = len(target2idx))

In [81]:
N_EPOCHS = 3
LR = 3e-3
WD = 0.8e-5

optimizer = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=WD)
criterion = nn.CrossEntropyLoss()

train_class = TrainingModule(model, optimizer, criterion, train_loader, val_loader, test_loader, N_EPOCHS, idx2target)
list_train_loss, list_val_loss, list_train_precision, list_val_precision,list_train_recall, list_val_recall, list_train_f1, list_val_f1 , list_train_accuracy,list_val_accuracy= train_class.train(dataset_name)


Epoch 1: train loss - 1.69194, validation loss - 1.36432
	 Validation: precision - 0.43548, recall - 0.43548, f1_score - 0.43548
Elapsed time: 2.828
----------------------------------------------------------------------
Epoch 2: train loss - 1.15529, validation loss - 1.41984
	 Validation: precision - 0.45161, recall - 0.45161, f1_score - 0.45161
Elapsed time: 3.836
----------------------------------------------------------------------
Epoch 3: train loss - 1.11151, validation loss - 1.35183
	 Validation: precision - 0.40323, recall - 0.40323, f1_score - 0.40323
Elapsed time: 6.320
----------------------------------------------------------------------


ValueError: not enough values to unpack (expected 10, got 8)

In [42]:

state_dict = torch.load( './' + dataset_name + '_article_model.pth')

model.load_state_dict(state_dict)

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = model.to(DEVICE)

In [43]:
import pandas as pd
from IPython.display import display

d = {'Original names': [], 'Predicted names': []}

for start, path, end, label in iter(test_loader):
    code, y_pred = model(start.to(DEVICE), path.to(DEVICE), end.to(DEVICE))
    y_pred = F.softmax(y_pred)
    y_pred = torch.argmax(y_pred, dim = 1)
    
    for i, j in zip(label, y_pred):
      d['Original names'].append(idx2target[i.item()])
      d['Predicted names'].append(idx2target[j.item()])
    break

df = pd.DataFrame(data=d)
display(df,)

/Users/unaissiddiqui/opt/anaconda3/envs/fyp/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  


,Original names,Predicted names
0,<unk>,7
1,7,7
2,2,10
3,7,7
4,5,10
...,...,...
57,7,7
58,10,7
59,7,7
60,7,10
